# Before labeling: PSTH correction

In [ ]:
# Prepare the toolbox

import random
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import find_peaks

In [ ]:
# Choose DataFrame

df = pd.read_csv("./data/psth_data_IC.csv").set_index("id").sort_index()
df

In [ ]:
# Adjust the time window of the PSTH

time_window_psth = df.columns[range(100,221)] # 0 to 120 ms

df_psth = df[time_window_psth]

In [ ]:
# Select a time window to control for the Spontaneous Firing Rate (SFR)

time_window_sfr = df.columns[range(100)] # -100 to 0 ms

df_psth_sfr = df[time_window_sfr]

In [ ]:
# Stablish a baseline to substract the SFR out of the PSTH of the evoked response

baseline = df_psth_sfr.mean(axis=1) + df_psth_sfr.std(axis=1) # baseline

In [ ]:
# Correct PSTH of the evoked responses by substracting the baseline

df_psth_corr = pd.DataFrame(index=df_psth.index, columns=df_psth.columns) # Prealocate a DataFrame for the corrected PSTHs

for col in df_psth.columns:
    df_psth_corr.loc[:,col] = df_psth.loc[:,col] - baseline

df_psth_corr[df_psth_corr<1e-50]=0 # DO I NEED THIS ANYMORE?
df_psth_corr

In [ ]:
# Check some examples to see that everything is OK

# Stimulus: pure tone (75 ms). Interstimulus interval of 250 ms (4 Hz presentation rate)
tone_x, tone_y = [0,75],[-0.05,-0.05] # info for Matplotlib to represent the tone under the PSTH

example = random.randrange(0,len(df_psth_corr.index)-1)
df_psth_corr.iloc[example].plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

# Heuristic Labeling

Now we are going to find some CLEAR EXAMPLES of each response type by establishing rather CONVERSATIVE CRITERIA.
Then we can use those labeled examples to evaluate the classification of different algorithms.
Response patterns:

    1) IRRESPONSIVE: plots showing no significant evoked response

- *PHASIC*: sharp responses evoked by the beginning and/or the end of a tone.

    2) ONSET: narrow PSTHs adjusted to the beginning of the stimulus.

    3) OFFSET: narrow PSTHs adjusted to the end of the stimulus.

    4) ON-OFF: evoked responses marking the beginning and the end of the stimulus.
    

- *TONIC*: long responses that follow the whole duration of the tone.

    5) PAUSER

    6) ON-SUSTAIN

    7) SUSTAIN

In [ ]:
# IMPORTANT to use copy() OR modifications done in df_pattern will propagate back to df_psth_corr!
df_pattern = df_psth_corr.copy() 
df_pattern['pattern']=''
df_pattern

# Choose thresholds. BE CONSERVATIVE at this stage!
noise_thr = 0.01 # Minimum to consider activity as an evoked response (remember: comes after baseline correction!).
weak_thr = 0.1 # To discard activity that might be too weak to be considered a proper response.
robust_thr = 0.25 # To select only robust responses.

### 1) IRRESPONSIVE pattern

In [ ]:
# Find some clearly IRRESPONSIVE patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    if psth.max() < noise_thr: # Time window could be adjusted (e.g., psth[0:121]), but BE CONSERVATIVE at this stage!
        df_pattern.loc[id,'pattern'] = "irresponsive"
        
print(len(df_pattern[df_pattern['pattern']=="irresponsive"]), "PSTHs have been labeled as 'irresponsive'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="irresponsive"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="irresponsive"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 2) ONSET pattern

In [ ]:
# Find some clear ONSET patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    if (
        psth[0:41].max() > weak_thr # Looks for an evoked response within the onset time window
    ) & (
        psth[41:-1].max() < noise_thr # Discards when there is activity passed the onset time window
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'onset'): # Overlap warning
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "onset" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="onset"]), "PSTHs have been labeled as 'onset'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="onset"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="onset"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 3) OFFSET pattern

In [ ]:
# Find some clear OFFSET patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    if (
        psth[0:76].max() < noise_thr # Discards when there is activity during the stimulation time window (75 ms)
    ) & (
        psth[76:121].max() > weak_thr # Looks for an evoked response within the offset time window
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'offset'): # Overlap warning
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "offset" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="offset"]), "PSTHs have been labeled as 'offset'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="offset"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="offset"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 4) ON-OFF pattern

In [ ]:
# Find some clear ON-OFF patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    if (
        psth[0:41].max() > weak_thr  # Looks for an evoked response within the onset time window
    ) & (
        psth[41:71].max() < noise_thr # Discards when there is activity between onset and offset time windows
    ) & (
        psth[71:101].max() > weak_thr  # Looks for an evoked response within the offset time window
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'on-off'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "on-off" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="on-off"]), "PSTHs have been labeled as 'on-off'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="on-off"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="on-off"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 5) PAUSER pattern

In [ ]:
# Find some clear PAUSER patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    peaks, _ = find_peaks(psth)
    peaks_in_tone = peaks[peaks<=70] # Peaks must be well within the duration of the stimulus (75 ms)
    if len(peaks_in_tone) < 2: # There must be at least 2 peaks within the tone to be considered a pauser.
        continue
    elif (
        psth[peaks_in_tone[0]] >= robust_thr # Looks for a robust onset component.
    ) & (
        psth[peaks_in_tone[-1]] >= weak_thr # Looks for a proper post-onset evoked response.
    ) & (
        psth[list(range(peaks_in_tone[0],peaks_in_tone[-1]))].min() < psth[peaks_in_tone[0]]/3 # DIP after 1st peak
    ) & (
        psth[list(range(peaks_in_tone[0],peaks_in_tone[-1]))].min() < psth[peaks_in_tone[-1]]/3 # DIP before last peak
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'pauser'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "pauser" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="pauser"]), "PSTHs have been labeled as 'pauser'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="pauser"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="pauser"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 6) ON-SUSTAIN pattern

In [ ]:
# Find some clear ON-SUSTAIN patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    peaks, _ = find_peaks(psth)
    peaks_in_tone = peaks[peaks<=70] # Peaks must be well within the duration of the stimulus (75 ms)
    if len(peaks_in_tone) < 2: # There must be at least 2 peaks within the tone to be considered a pauser.
        continue
    elif (
        peaks_in_tone[0] < 31 # Checks if the 1st peak is within the onset time window.
    ) & (
        psth[peaks_in_tone[0]] >= weak_thr # Checks if that 1st peak is robust.
    ) & (
        psth[peaks_in_tone[-1]] >= noise_thr # Checks if that last peak is high enough.
    ) & (
        psth[peaks_in_tone[-1]] < psth[peaks_in_tone[0]]/2 # Checks if the last peak is at least half of the 1st peak.
    ) & (
        psth[peaks_in_tone[0]:peaks_in_tone[-1]].min() > psth[peaks_in_tone[0]]/3 # Checks sustain activity between peaks
    ) & (
        psth[peaks_in_tone[0]:70].min() > noise_thr # Discards any pause during the stimulus presentation
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'on-sustain'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "on-sustain" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="on-sustain"]), "PSTHs have been labeled as 'on-sustain'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="on-sustain"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="on-sustain"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

### 7) SUSTAIN pattern

In [ ]:
# Find some clear SUSTAIN patterns

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    peaks, _ = find_peaks(psth)
    peaks_in_tone = peaks[peaks<=50] # Peaks must be well within the duration of the stimulus (75 ms)
    if len(peaks_in_tone) == 0:
        continue
    elif (
        psth[peaks_in_tone[0]] >= robust_thr # Checks that the 1st peak is robust
    ) & (
        psth[peaks_in_tone[0]:71].min() > psth[peaks_in_tone[0]] - weak_thr # Checks that sustain response is rather flat
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'sustain'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_pattern.loc[id,'pattern'] = "sustain" # Assigns ONSET label
        
print(len(df_pattern[df_pattern['pattern']=="sustain"]), "PSTHs have been labeled as 'sustain'")

In [ ]:
# Check some examples to see that everything is OK

df_pattern[df_pattern['pattern']=="sustain"].iloc[random.randrange(
    len(df_pattern[df_pattern['pattern']=="sustain"]))].drop('pattern').plot(kind='line')
plt.ylim([-0.1, 1])
plt.xlabel('Time (ms)')
plt.ylabel('Spike density (norm)')
tone, = plt.plot(tone_x, tone_y, marker = 'o')
tone.set_label('Tone')
plt.legend()
plt.show()

# Preliminar classification report

In [ ]:
# Preliminar classification report

display(df_pattern['pattern'].value_counts().drop(''))
print(sum(df_pattern['pattern'] != ''), "recordings met the criteria to be labeled in one category.")
print(round(sum(df_pattern['pattern'] == '')*100/len(df_pattern['pattern']),1), "% of the recordings remain unlabeled.")

In [ ]:
# Save DataFrame to a csv
df_pattern.to_csv("./data/psth_data_IC_preclassified.csv")

# Drafting area... (IGNORE!)

In [ ]:
# Find some clear SUSTAIN patterns

i=0
for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    peaks, _ = find_peaks(psth)
    peaks_in_tone = peaks[peaks<=50] # Peaks must be well within the duration of the stimulus (75 ms)
    if len(peaks_in_tone) == 0:
        continue
    elif (
        psth[peaks_in_tone[0]] >= weak_thr # Checks that the 1st peak is robust
    ) & (
        psth[peaks_in_tone[0]].min() >= weak_thr # Checks that sustain response is rather flat
    ) & (
        psth[peaks_in_tone[0]:71].min() > psth[peaks_in_tone[0]] - weak_thr # Checks that sustain response is rather flat
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'sustain'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_psth_corr.loc[id].plot(kind='line')
            plt.title(id)
            plt.ylim([-0.1, 1])
            plt.xlabel('Time (ms)')
            plt.ylabel('Spike density (norm)')
            tone, = plt.plot(tone_x, tone_y, marker = 'o')
            tone.set_label('Tone')
            plt.legend()
            plt.show()
            i+=1

print(i,"recordings labeled.")

In [ ]:
# Find some clear ON-SUSTAIN patterns

i=0

for id in df_psth_corr.index:
    psth = df_psth_corr.loc[id]
    peaks, _ = find_peaks(psth)
    peaks_in_tone = peaks[peaks<=70] # Peaks must be well within the duration of the stimulus (75 ms)
    if len(peaks_in_tone) < 2: # There must be at least 2 peaks within the tone to be considered a pauser.
        continue
    elif (
        peaks_in_tone[0] < 31 # Checks if the 1st peak is within the onset time window.
    ) & (
        psth[peaks_in_tone[0]] >= weak_thr # Checks if that 1st peak is robust.
    ) & (
        psth[peaks_in_tone[-1]] >= noise_thr # Checks if that last peak is high enough.
    ) & (
        psth[peaks_in_tone[0]]/1.2 > psth[41:70].max() # Checks if the last peak is at least half of the 1st peak.
    ) & (
        psth[peaks_in_tone[0]:peaks_in_tone[-1]].min() > psth[peaks_in_tone[0]]/3 # Checks sustain activity between peaks
    ) & (
        psth[peaks_in_tone[0]:70].min() > noise_thr # Discards any pause during the stimulus presentation
    ):
        if (df_pattern['pattern'].loc[id] != '') & (df_pattern['pattern'].loc[id] != 'on-sustain'):
            print("OVERLAPPING CRITERIA!", id, "has already been classified as", df_pattern['pattern'].loc[id])
        else:
            df_psth_corr.loc[id].plot(kind='line')
            plt.title(id)
            plt.ylim([-0.1, 1])
            plt.xlabel('Time (ms)')
            plt.ylabel('Spike density (norm)')
            tone, = plt.plot(tone_x, tone_y, marker = 'o')
            tone.set_label('Tone')
            plt.legend()
            plt.show()
            i+=1

print(i,"recordings labeled.")

In [ ]:
psth[list(range(peaks_in_tone[0],peaks_in_tone[-1]))].min() < psth[peaks_in_tone[0]]/3